# Use the data from Classifier and find the vehicle

In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

# load params and classifier from pickled file
scv = None
X_scaler = None
orient = None
pix_per_cell = None
cell_per_block = None
spatial_size = None
hist_bins = None
color_space = None

with open('./svc_pickle.p', 'rb') as f:
    data_pickle = pickle.load(f)
    svc = data_pickle['svc'] 
    X_scaler = data_pickle['scaler'] 
    orient = data_pickle['orient'] 
    pix_per_cell = data_pickle['pix_per_cell'] 
    cell_per_block = data_pickle['cell_per_block'] 
    spatial_size = data_pickle['spatial_size'] 
    hist_bins = data_pickle['hist_bins'] 
    color_space = data_pickle['color_space']

#Take the data from Classifier
    
def color_hist(img, nbins=128, bins_range=(0, 256)):
    channel1 = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2 = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3 = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    features = np.concatenate((channel1[0], channel2[0], channel3[0]))
    return features

def bin_spatial(img, size=(16, 16)):
    resize_img = cv2.resize(img, size)
    color1 = resize_img[:,:,0].ravel()
    color2 = resize_img[:,:,1].ravel()
    color3 = resize_img[:,:,2].ravel()
    features = np.hstack((color1, color2, color3))
    return features

def get_hog_features(img, orient=9, pix_per_cell=8, cell_per_block=2, feature_vec=False):
    return hog(
        img, orient, 
        pixels_per_cell=(pix_per_cell, pix_per_cell),
        cells_per_block=(cell_per_block, cell_per_block),
        visualise=False, feature_vector=feature_vec)

# sliding window search

def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    for bbox in bboxes:
        top_left = bbox[0]
        bottom_right = bbox[1]
        cv2.rectangle(imcopy, (top_left[0], top_left[1]), (bottom_right[0], bottom_right[1]), color, thick)
    return imcopy

#search the window

def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list